In [1]:
from models import load_all_models
model_lst = load_all_models()

Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device cuda with dtype torch.float16
Loaded reading order model vikp/surya_order on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded texify model to cuda with torch.float16 dtype
Loaded recognition model vikp/surya_tablerec on device cuda with dtype torch.float16


In [2]:
from convert import convert_single_pdf
fpath = "/home/jenny/Downloads/Archive 2/scm_metals/NCS000050_Permit (Issuance)_20230524.pdf"
fpath = "/home/jenny/Downloads/Archive/wilders_grove_yard_waste/WI0500447/WI0500447_Correspondence_20190529.pdf"
# fpath = "/home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Staff Report_20200911.pdf"
# fpath = "/home/jenny/Downloads/01005_Eakes Drycleaners_BF Site Assmt.pdf"
full_text, doc_images, out_meta, pages, text_blocks = convert_single_pdf(fpath, model_lst)

Recognizing Text:   0%|          | 0/1 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.51 GiB. GPU 0 has a total capacity of 23.55 GiB of which 1.46 GiB is free. Process 10966 has 1.04 GiB memory in use. Process 52004 has 5.96 GiB memory in use. Including non-PyTorch memory, this process has 15.02 GiB memory in use. Of the allocated memory 12.53 GiB is allocated by PyTorch, and 2.03 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [19]:
[[block.id, block.bbox, block.text, block.block_type, block.page_end] for i, block in enumerate(text_blocks)]

[['1|-7',
  [184.0, 94.04461319411486, 728.0, 679.3222591362127],
  'Division of Waste Management December 23, 1997 Mr. Phil Vorsatz\n\n![0_image_0.png](0_image_0.png)\n\n\n\n![0_image_1.png](0_image_1.png)\n\n\n\n![0_image_2.png](0_image_2.png)\n\n\nNC CERCLA Project Officer EPA Region IV Waste Division 345 Courtland Street, NE\nAtlanta, Georgia 30365 RE:\nBrownfield Site Assessment Report Eakes Dry Cleaners.\n\nDurham, Durham County, North Carolina Dear Mr. Vorsatz:\nThis letter is being sent to you to convey the information that the North Carolina Superfund Section has obtained on a site named Eakes Dry Cleaners.\n\nEakes Cleaners is located at 827 West Morgan Street in Durham, North Carolina.  It consists of a 6,600 square foot brick and block building situated on approximately 0.3 acres. The site was a Dry Cleaning and Fur Storage business for many years until its closure in 1995, after the death of Mr. Eakes and the bankruptcy of his estate. The site is located in a mixed commerc

In [9]:
import os
allfn = os.listdir("/home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447")
pdfs = set([fn for fn in allfn if fn.endswith(".pdf")])
mds = set([fn for fn in allfn if not fn.endswith(".pdf")])

In [16]:
import os
from convert import convert_single_pdf
from marker.schema.page import Page
from output import save_markdown
import pandas as pd

def page_to_dict(page: Page) -> dict:
    # Manually convert Page to dictionary if necessary
    return {
        'bbox':page.bbox,
        'pnum': page.pnum,
        'rotation': page.rotation,
        'text_lines': page.text_lines,  # Assuming text_lines can be converted
        'layout': page.layout,          # Assuming layout can be converted
        'order': page.order,
        'ocr_method': page.ocr_method,
        'char_blocks': page.char_blocks,  # Assuming lists of dicts can be converted
        'images': page.images,           # Handle images if needed
    }
def get_dir_pdfs(parent_dir):
    if not os.path.exists(parent_dir):
        raise Exception("Starting directory '%s' does not exist" % parent_dir)
    return [fn for fn in os.listdir(parent_dir) if ".pdf" in fn]

def to_refdocid(s):
    return (
        s.replace(".pdf","").replace(" ", "_")
        .replace("(", "")
        .replace(")", "")
        .replace("/", "_")
        .replace(".", "-")
        .replace("#", "")
        .replace("'", "")
    )
def rename_pdf(parent_dir, filename, new_filename):
    old_file = os.path.join(parent_dir, filename)
    new_file = os.path.join(parent_dir, new_filename)
    os.rename(old_file, new_file)
    
def run(parent_dir, output_fn):
    pdf_fns = get_dir_pdfs(parent_dir)
    all_tblocks = []
    all_pages = []
    error_files = []
    for fn in pdf_fns:
        docid = to_refdocid(fn)
        rename_pdf(parent_dir, fn, docid + ".pdf")
        input_path = os.path.join(parent_dir, docid + ".pdf")
        try:
            full_text, images, out_meta, pages, tblocks = run_single(input_path, fn)
            subfolder_path = save_markdown(parent_dir, fn, full_text, images, out_meta)
            pd.DataFrame(tblocks, columns=['id', 'bbox', 'text', 'type', 'page_end']).to_csv(f"{subfolder_path}/{fn}_blocks.csv")
            page_data = [page_to_dict(page) for page in pages]
            pd.DataFrame(page_data).to_csv(f"{subfolder_path}/{docid}_pages.csv")
            all_pages.append([docid, page_data[0]['bbox']])
            pd.DataFrame(all_pages).to_csv(f"{output_fn}_pages.csv")
            print(f"Processed {len(tblocks)} blocks for {fn}")
            all_tblocks.extend(tblocks)
            pd.DataFrame(all_tblocks, columns=['id', 'bbox', 'text', 'type', 'page_end']).to_csv(f"{output_fn}_blocks.csv")
        except Exception as e:
            print(f"Error processing {input_path}: {e}")
            error_files.append(fn)
    return all_tblocks, all_pages, error_files

def run_single(input_path, fn):
    full_text, images, out_meta, pages, text_blocks = convert_single_pdf(input_path, model_lst)
    tblocks = [[f"{fn}-|{block.id}", block.bbox, block.text, block.block_type, block.page_end] for i, block in enumerate(text_blocks)]
    return full_text, images, out_meta, pages, tblocks

In [19]:
all_blocks, all_pages, errors = run("/home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447", "WI0500447")

Recognizing Text:  10%|█         | 3/29 [00:12<01:47,  4.14s/it]

Error processing /home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Application_20190523.pdf: CUDA out of memory. Tried to allocate 3.51 GiB. GPU 0 has a total capacity of 23.55 GiB of which 81.88 MiB is free. Process 11776 has 1.61 GiB memory in use. Process 10966 has 1.04 GiB memory in use. Including non-PyTorch memory, this process has 20.75 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 5.80 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)



Recognizing Text:   3%|▎         | 4/125 [00:17<08:53,  4.41s/it]


Error processing /home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Report_20190311.pdf: CUDA out of memory. Tried to allocate 3.49 GiB. GPU 0 has a total capacity of 23.55 GiB of which 2.07 GiB is free. Process 10966 has 1.04 GiB memory in use. Including non-PyTorch memory, this process has 20.38 GiB memory in use. Of the allocated memory 14.41 GiB is allocated by PyTorch, and 5.51 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Detecting bboxes:  28%|██▊       | 22/79 [00:16<00:43,  1.30it/s]

...

Error processing /home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Report_20150605.pdf: CUDA out of memory. Tried to allocate 3.49 GiB. GPU 0 has a total capacity of 23.55 GiB of which 2.80 GiB is free. Process 10966 has 1.04 GiB memory in use. Including non-PyTorch memory, this process has 19.65 GiB memory in use. Of the allocated memory 14.34 GiB is allocated by PyTorch, and 4.85 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)



Detecting bboxes:   3%|▎         | 5/144 [00:05<02:33,  1.11s/it]

Error processing /home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Monitoring_Report_20200612.pdf: list indices must be integers or slices, not NoneType



Finding reading order: 100%|██████████| 2/2 [00:00<00:00,  2.94it/s]
Recognizing tables: 0it [00:00, ?it/s]


Processed 17 blocks for WI0500447_Site_Plans_Maps_20150605_3.pdf


In [ ]:
import pymupdf4llm
md_text = pymupdf4llm.to_markdown("/home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Application_20190523.pdf")
md_text

In [12]:
df = pd.read_csv("all_tblocks.csv")
df.head(2)

,Unnamed: 0,id,bbox,text,type,page_end
0,0,01005_Eakes_Drycleaners_BF_Site_Assmt.pdf-|1|-18,"[60.02847650688182, 78.0, 667.3165638348363, 6...",Environment and Natural Resources North Caroli...,Text,False
1,1,01005_Eakes_Drycleaners_BF_Site_Assmt.pdf-|1|-33,"[664.3151400094922, 382.0, 1027.4874228761273,...","36° 00' 02.5"" and West 078° 54' 36.0"".\n\nOn O...",Text,True


In [ ]:
# NUM_DEVICES=2 DEBUG=true marker_chunk_convert /home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Application_20190523.pdf /home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Application_20190523

In [16]:
df.text.to_list()

['Environment and Natural Resources North Carolina Department of Division of Waste Management December 23, 1997 Mr. Phil Vorsatz\n\n![0_image_0.png](0_image_0.png)\n\n\nRCLA Project Officer Region IV Waste Division 5 Courtland Street, NE\nAtlanta, Georgia 30365 RE\nDear Mr. Vorsatz:\nBrownfield Site Assessment Report Eakes Dry Cleaners,\nDurham, Durham County, North Carolina industrial, and residential area in downtown Durham.  The site is located at North approximately 0.3 acres.   The site was a Dry Cleaning and Fur Storage business\nfor many years until its closure in 1995, after the death of Mr. Eakes and the Carolina.  It consists of a 6,600 square foot brick and block building situated on Eakes Cleaners is located at 827 West Morgan Street in Durham, North This letter is being sent to you to convey the information that the North Carolina Superfund Section has obtained on a site named Eakes Dry Cleaners.\n\nThe site is located in a mixed commercial, light pankruptcy of his estate.

In [15]:
df[df.id.str.contains("01005_Eakes_Drycleaners_BF_Site_Assmt")].bbox.tolist()

['[60.02847650688182, 78.0, 667.3165638348363, 620.0]',
 '[664.3151400094922, 382.0, 1027.4874228761273, 117.0]',
 '[86.04081632653062, 341.0, 107.05078310393925, 377.0]',
 '[213.89848197343457, 230.0, 1143.4573055028463, 1198.0]',
 '[569.2700522069293, 467.0, 625.296630280019, 377.0]',
 '[51.0, 14.0, 928.0, 163.0]',
 '[460.0, 164.0, 495.0, 180.0]',
 '[310.0, 218.0, 959.0, 322.0]',
 '[243.0, 410.0, 329.0, 463.0]',
 '[112.0, 334.0, 1029.0, 516.0]',
 '[23.0, 1583.0, 1042.0, 1534.0]',
 '[642.0, 442.0, 681.0, 575.0]',
 '[710.0, 632.0, 824.0, 814.0]',
 '[80.0, 32.0, 1262.0, 298.0]',
 '[295.0, 1114.0, 755.0, 1365.0]',
 '[909.0, 102.0, 990.0, 97.0]',
 '[246.0, 242.0, 940.0, 1174.0]',
 '[470.0, 419.0, 834.0, 372.0]',
 '[94.0, 108.0, 399.0, 622.0]',
 '[398.0, 153.0, 703.0, 578.0]',
 '[702.0, 151.0, 799.0, 294.0]',
 '[818.0, 221.0, 846.0, 294.0]',
 '[847.0, 137.0, 895.0, 294.0]',
 '[876.0, 606.0, 891.0, 627.0]',
 '[876.0, 606.0, 891.0, 627.0]']

In [17]:
page_df = pd.read_csv("/home/jenny/Downloads/Archive 2/scm_metals/NCS000050_Inspection_Report_20090917/NCS000050_Inspection_Report_20090917.pdf_pages.csv")
page_df.head(2)

,Unnamed: 0,pnum,rotation,text_lines,layout,order,ocr_method,char_blocks,images
0,0,0,0,"bboxes=[PolygonBox(polygon=[[273, 55], [525, 5...","bboxes=[LayoutBox(polygon=[[691, 991], [761, 9...","bboxes=[OrderBox(bbox=[691.0, 991.0, 761.0, 10...",NaN,"[{'lines': [{'spans': [{'font': {'size': 9.0, ...",[]
1,1,1,0,"bboxes=[PolygonBox(polygon=[[114, 53], [221, 5...","bboxes=[LayoutBox(polygon=[[688, 990], [762, 9...","bboxes=[OrderBox(bbox=[688.0, 990.0, 762.0, 10...",NaN,"[{'lines': [{'spans': [{'font': {'size': 5.0, ...",[]


In [6]:
df.type.value_counts()

type
Text              1576
Section-header     513
Figure             270
Table              269
Title              139
Formula             86
List-item           48
Caption             30
Code                29
Page-header         13
Page-footer          4
Footnote             3
Name: count, dtype: int64

In [ ]:
df[(df.type)]